In [ ]:
from pocket_cube.cube import Cube
from pocket_cube.cube import Move
from tests import test_list, test, is_solved
from heuristics import hamming, inverse_hamming, manhattan
from utils import get_neighbors, get_path, met_in_the_middle, FrontierItem, DiscoveredDict

import numpy as np
from heapq import heappush, heappop
from typing import Callable

%matplotlib notebook


 # A*
 initialize with cube
 search
 get path

 # Test A*

 # Bidirectional BFS

 # Test Bidirectional BFS

 # MTCS with UCB

In [ ]:
from math import sqrt, log

def select_move(node: Node, c) -> Move:
    N_node = node[N]
    max_move: Move = None
    max_expr: float = float('-inf')
    for move in node[CHILDREN]:
        child = node[CHILDREN][move]
        expr = child[Q] / child[N] + c * sqrt(log(N_node) / child[N])
        if expr > max_expr:
            max_expr = expr
            max_move = move
    return max_move

In [ ]:
from random import choice

def mcts(cube0: Cube, budget: int, tree: Node, cp: float, heuristic: Callable[[Cube], int]) -> Node:
    if not tree:
        tree = init_node()
    for _ in range(budget):
        cube = cube0
        node = tree
        # go down the tree until a final state or an unexplored move is found
        while not is_solved(cube) and not any([move not in node[CHILDREN] for move in Move]):
            move: Move = select_move(node, cp)
            cube = cube.move(move)
            node = node[CHILDREN][move]
        # if node is not final and not every move has been explored, create a new node
        if not is_solved(cube):
            new_node: Node = init_node(node)
            move: Move = choice([move for move in Move if move not in node[CHILDREN]])
            node[CHILDREN][move] = new_node
            cube = cube.move(move)
            node = new_node
        # simulate a random game
        max_moves: int = 14
        max_h: int = 0
        while not is_solved(cube) and max_moves > 0:
            new_node: Node = init_node(node)
            move: Move = choice([move for move in Move])
            node[CHILDREN][move] = new_node
            cube = cube.move(move)
            node = new_node
            max_h = max(max_h, heuristic(cube))
            max_moves -= 1
        while node:
            node[N] += 1
            node[Q] += max_h
            node = node[PARENT]
    return tree

def play_mtcs(cube: Cube, budget: int, cp: float, heuristic: Callable[[Cube], int]) -> list[Move]:
    tree: Node = mcts(cube, budget, None, cp, heuristic)
    node: Node = tree
    path: list[Move] = []
    while node and node[CHILDREN]:
        move: Move = select_move(node, 0)
        node = node[CHILDREN][move]
        path.append(move)
    return path

 # Test MTCS